This cookbook will guide you through the creation of a simple visualization from an existing trial in jinko. In particular, you will be able to retrive timeseries and plot them using plotly.

It uses the trial [tr-OxkW-mB8I](https://jinko.ai/tr-OxkW-mB8I), located in the common [API cookbooks project](https://jinko.ai/project/e0fbb5bb-8929-439a-bad6-9e12d19d9ae4).

In [ ]:
# Initialize your connection to Jinko

import sys; sys.path.insert(0, '../lib')
import jinko_helpers as jinko

# Configuration

# Fill your API key (ex: 'd74cc07e-4a86-4aab-952a-a5814ae6e1ee')
# If not set, it will be retrieved from the JINKO_API_KEY environment variable.
# If environment variable is not set, you will be asked for it interactively
apiKey = None

# Fill your Project Id (ex: '14495063-918c-441a-a898-3131d70b02b0')
# If not set, it will be retrieved from the JINKO_PROJECT_ID environment variable.
# If environment variable is not set, you will be asked for it interactively
projectId = None

# This function ensures that authentication is correct
# It it also possible to override the base url by passing baseUrl=...
jinko.initialize(projectId, apiKey = apiKey)

The first step will be to fetch the trial outputs and the data we want to plot.

In [ ]:
import zipfile
import io
from typing import List, Any
import json

# Fill the short Id of your Trial
trialShortId = 'tr-OxkW-mB8I' # edit this line if necessary
if trialShortId is None:
  raise Exception('Please specify a Trial Id')

# Convert short Id to coreItemId
coreItemId = jinko.getCoreItemId(trialShortId, 1)

# List all Trial versions
versions: List[Any] = jinko.makeRequest('/core/v2/trial_manager/trial/%s/status'%(coreItemId['id'])).json()

# Get the latest completed version
latestCompletedVersion = next((item for item in versions if item['status'] == 'completed'), None)
if latestCompletedVersion is None:
  raise Exception('No completed Trial version found')

print("Successfully fetched this simulation:\n",json.dumps(latestCompletedVersion,indent=1))

# Store the trial Id and the snapshot Id to use in the API requests
simulationId = latestCompletedVersion['simulationId']
trialId = simulationId['coreItemId']
trialSnapshotId = simulationId['snapshotId']

In [ ]:
# Retrieve results summary
response = jinko.makeRequest('/core/v2/trial_manager/trial/%s/snapshots/%s/results_summary'%(trialId,trialSnapshotId), method='GET')
responseSummary = json.loads(response.content)

# Print a summary of the results content
print("Keys in the results summary:\n",list(responseSummary.keys()),"\n")
print("Available patients:\n",responseSummary['patients'],"\n")
print("Available arms:\n",responseSummary['arms'],"\n")
print("Available scalars:\n",[scalar['id'] for scalar in responseSummary['scalars']],"\n")
print("Available cross-arm scalars:\n",[scalar['id'] for scalar in responseSummary['scalarsCrossArm']],"\n")
print("Available categoricals:\n",[scalar['id'] for scalar in responseSummary['categoricals']],"\n")
print("Available cross-arm categoricals:\n",[scalar['id'] for scalar in responseSummary['categoricalsCrossArm']],"\n")

# Store the list of scenario descriptors fetch them
scenarioDescriptors = [scalar['id'] for scalar in (responseSummary['scalars']+responseSummary['categoricals']) if "ScenarioOverride" in scalar['type']['labels']]
print("List of scenario overrides:\n",scenarioDescriptors,"\n")

In [ ]:
# Retrieve timeseries ids
response = jinko.makeRequest('/core/v2/trial_manager/trial/%s/snapshots/%s/output_ids'%(trialId,trialSnapshotId), method='GET')
responseSummary = json.loads(response.content.decode("utf-8"))
print("Available timeseries:\n",responseSummary,"\n")
# Store the list of time series to fetch them later
timeseriesList=responseSummary

In [ ]:
# Retrieve time series
response = jinko.makeRequest('/core/v2/result_manager/timeseries_summary', method='POST'
    , json={
      "select": timeseriesList,
      "trialId": simulationId
    })
# Timeseries are stored in a zip file
archive = zipfile.ZipFile(io.BytesIO(response.content))
filename = archive.namelist()[0]
print("Successfully fetched:\n",filename)

csvTimeseries = archive.read(filename).decode('utf-8')

In [ ]:
# Retrieve scalar results containing scenario overrides
response = jinko.makeRequest('/core/v2/result_manager/scalars_summary', method='POST', json={
  "select": scenarioDescriptors,
  "trialId": simulationId
})
archive = zipfile.ZipFile(io.BytesIO(response.content))
filename = archive.namelist()[0]
print("Successfully fetched:\n",filename)

csvScalars = archive.read(filename).decode('utf-8')


In [ ]:
We will now process the data using the pandas library, and transform it into a table taht can easily be plotted.

In [ ]:
import pandas as pd

# Load timeseries into a dataframe
dfTs = pd.read_csv(io.StringIO(csvTimeseries))
print("Raw timeseries data : \n",dfTs[1:3],"\n")

# Load scalars into a dataframe
dfScalars = pd.read_csv(io.StringIO(csvScalars))
print("Raw scalar data  :\n",dfScalars[1:3],"\n")

# Pivot to a wide format to obtain protocol overrides in columns
dfScalars = dfScalars.pivot(index = ["armId","patientId"], columns="scalarId", values = "value")

print("Pivotted scalar table: \n")
print(dfScalars[1:3])
print("\n")

# Merge both tables together to obtain protocol descriptors per arm
df = dfTs.merge(right=dfScalars,left_on=["Patient Id","Arm"],right_on=["patientId","armId"])

print("Merged table: \n")
print(df[1:3])
print("\n")

# Convert time to days
df["Time"]=df["Time"].map(lambda x: x / (60 * 60 * 24))

# Filter the data set to keep only the variable we would like to plot
dfToPlot = df.query('Descriptor=="Blood.Drug"')

In [ ]:
Finally we plot the time series data by facetting over scenario overrides.

In [ ]:
import plotly.express as px
import plotly.io as pio
# The renderer needs to be adjusted depending on the use case
# pio.renderers.default = 'iframe' # for running in jupyter lab
pio.renderers.default = 'vscode' # for running in vs code

fig = px.line(dfToPlot,
              x="Time",
              y="Value",
              facet_col="fullDose.tmin", 
              facet_row="primingDose.tmin",
              color="administrationMode",
              labels={
                "Time": "Time (days)",
                "Value": "Concentration (µg/mL)",
                "fullDose.tmin": "Full dose (mg)",
                "primingDose.tmin": "Priming dose (mg)",
                "administrationMode": "Administration"
                },
              log_y=True
             )
fig.show()